# III. 반도체 박막 모델 업그레이드  
모델을 수정하여 손실값을 더 줄여 본다.

---
### 1) 데이터 로드 : 전 단계에서 준비한 트레이닝용 데이터를 로드하여 모델 트레이닝에 사용

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 경고 메시지 안보이게 설정

import gc
gc.collect() # garbage collector : 메모리 관리

In [ ]:
import os
import tensorflow as tf
import tensorflow.keras as keras

---
전역 변수 중 일부(디렉토리 이름과 파일 이름 등)는 대문자로  
나머지 변수는 소문자로

In [ ]:
# 파일 경로는 단순히 문자열 연결보다는 os.path.join()을 사용하는 것이 좋음 
DATA_DIR = 'data'
TRAIN_DATA_FILE = 'train-splited.csv'
TRAIN_DATA_PATH = os.path.join(DATA_DIR, TRAIN_DATA_FILE)

In [ ]:
import pandas as pd

train_df = pd.read_csv(TRAIN_DATA_PATH)

---
### 2) 성능을 향상시키디 위하여 네트워크 모델을 개선

In [ ]:
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation

import numpy as np

---
#### (가) 사용자 정의 활성함수(Activation Function) 추가 : gelu()

In [ ]:
def gelu(x):  return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

In [ ]:
get_custom_objects().update({'gelu': Activation(gelu)})

---
#### (나) 네트워크 모델 만들기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
# 새로운 Sequential 모델을 정의합니다
model = Sequential([
  Dense(units=1024, activation='gelu', input_dim=226),
  Dense(units=900, activation='gelu'),
  Dropout(0.02),
  Dense(units=1024, activation='gelu'),
  Dense(units=512, activation='gelu'),
  Dense(units=512, activation='gelu'),
  Dense(units=4, activation='linear')
])

In [ ]:
# 모델을 컴파일합니다.
model.compile(loss='mae', optimizer='adam', metrics=['mae'])

In [ ]:
model.summary()

In [ ]:
MODEL_DIR = 'model'
os.makedirs(MODEL_DIR, exist_ok=True)

In [ ]:
MODEL_SUMMARY_FILE = "thin_film_base_model.png"
MODEL_SUMMARY_PATH = os.path.join(MODEL_DIR, MODEL_SUMMARY_FILE)

from tensorflow.keras.utils import plot_model

plot_model(model, to_file = MODEL_SUMMARY_PATH, show_shapes=True)

---
#### (다) 체크포인트를 저장하기 콜백 만들기 : 에포크마타 모델 파일을 저장하기

In [ ]:
# MODEL_DIR = 'model'

from pathlib import Path

CHK_POINT_DIR = os.path.join(MODEL_DIR, 'upgrade_chk_point')
Path(CHK_POINT_DIR).mkdir(parents=True, exist_ok=True)
# os.makedirs(CHK_POINT_DIR, exist_ok=True)

CHK_POINT_FILE = '{epoch:002d}.h5'
CHK_POINT_PATH =  os.path.join(CHK_POINT_DIR, CHK_POINT_FILE)

In [ ]:
CHK_POINT_PATH

In [ ]:
# callback
checkpoint = keras.callbacks.ModelCheckpoint(
    CHK_POINT_PATH,
    monitor='val_loss',
    verbose=1,
    save_best_only=False,
    mode='min',
    save_weights_only=False
)

---
#### (라) 텐서보드용 로그 저장 콜백 만들기

In [ ]:
# MODEL_DIR = 'model'
# from pathlib import Path

TB_DIR = os.path.join(MODEL_DIR, 'tensorboard')
Path(TB_DIR).mkdir(parents=True, exist_ok=True)
# os.makedirs(CHK_POINT_DIR, exist_ok=True)

In [ ]:
# callback
tensorboard = keras.callbacks.TensorBoard(
    log_dir=TB_DIR,
    histogram_freq=0,
    write_graph=True,
    write_images=True,
)

---
#### (마) 러닝레이트 조정 콜백 만들기  
손실 값이 잘 줄지 않을 때 러닝레이트를 더 작게 줄인다.

In [ ]:
# callback
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=10,
    verbose=1,
    mode='auto',
    # epsilon=1e-04,
    min_delta=1e-04,
    cooldown=0,
    min_lr=0
)

In [ ]:
# 사용할 콜백을 콜맥 리스트에 저장
callbacks = [checkpoint, tensorboard, reduce_lr]

In [ ]:
# batch_size 2000일 때, NUC에서 epoch 당 약 135초
# 5 epoch : mae = 15.1 (12분)
# 10 epoch : mae = 6.8 (24분)
# 15 epoch : mae = 4.5 (36분)
# 20 epoch : mae = 3.5 (48분)


# batch_size 1000일 때, NUC에서 epoch 당 약 151초
# 5 epoch : mae = 7.5 (13분)

# batch_size 500일 때, NUC에서 epoch 당 약 183초
# 5 epoch : mae = 4.8 (15분)

# batch_size 250일 때, NUC에서 epoch 당 약 240초
# 5 epoch : mae = 4.45 (20분)

# batch_size 200일 때, NUC에서 epoch 당 약 260초
# 4 epoch : mae = 5.3 ()
# 5 epoch : mae = 9.7 (1314 초)

epoch_num = 5
batch_size = 250 # 200 # 250 # 500 # 1000 # 2000

In [ ]:
#독립변수와 종속변수를 분리합니다.
train_X = train_df.iloc[:, 5:]
train_Y = train_df.iloc[:, 1:5]

In [ ]:
import time

In [ ]:
#모델을 학습합니다.
start = time.time() # start time

history = model.fit(train_X, train_Y, epochs=epoch_num, batch_size=batch_size, validation_split=0.05, callbacks=callbacks)

print("time : ", time.time() - start, " sec") # sec

In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# loss 측정값의 시각화.  

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(loss)+1)

plt.plot(epochs,loss,label='Training Loss')
plt.plot(epochs,val_loss,label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()